![ups logo](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png/640px-Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png)

## Aprendizaje Automático P64

## Practica 04


### Árboles de decisión, Random Forest (RF) y Máquinas de Soporte Vectorial (SVM)

### Fase 4: Predicciones con Maquina de Soporte Vectorial / Support Vector Machine (SVM)

### Autor: Diego Tapia

### Importación de librerías
<a id="importacion-de-librerias"></a>

In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 

from time import time
from datetime import datetime, timedelta

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle
import seaborn as sns
import copy

from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout

import keras
from keras.models import model_from_json
from keras.models import load_model

from sklearn import metrics 
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn import set_config   
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC

print("Módulos y clases importados")

2024-06-19 21:47:47.801657: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 21:47:49.014365: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Módulos y clases importados


### Carga del dataset

Se carga el dataset procesado de la fase 1

In [2]:
dataframe = pd.read_csv('dataset/dataset_processed_heart_failure.csv', sep=',', header=0)

dataframe.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,Y
0,55.0,0,748,0,45,0,263358.03,1.3,137,male,1,88,0
1,65.0,0,56,0,25,0,305000.00,5.0,130,male,0,207,0
2,45.0,0,582,1,38,0,319000.00,0.9,140,female,0,244,0
3,60.0,1,754,1,40,1,328000.00,1.2,126,male,0,90,0
4,95.0,1,582,0,30,0,461000.00,2.0,132,male,0,50,1


Se extrae la variable de salida 'Y'

In [3]:
Y = dataframe['Y']
dataframe.drop(['Y'], axis=1, inplace=True)
dataframe.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,55.0,0,748,0,45,0,263358.03,1.3,137,male,1,88
1,65.0,0,56,0,25,0,305000.00,5.0,130,male,0,207
2,45.0,0,582,1,38,0,319000.00,0.9,140,female,0,244
3,60.0,1,754,1,40,1,328000.00,1.2,126,male,0,90
4,95.0,1,582,0,30,0,461000.00,2.0,132,male,0,50


In [6]:
X = copy.deepcopy(dataframe)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("Conjunto de train y test preparado!")

Conjunto de train y test preparado!


### Carga del Pipeline

Se carga el pipeline obtenido en la fase 1

In [4]:
def cargarPipeline(nombreArchivo):
    with open(nombreArchivo+'.pickle', 'rb') as handle:
        pipeline = pickle.load(handle)
    return pipeline

In [5]:
nombreArchivoPreprocesador='pipeline_heart_failure'
pipe=None
pipe=cargarPipeline(nombreArchivoPreprocesador) #Debes incluir el archivo del Pipeline en el mismo directorio del script 
cantidadPasos=len(pipe.steps)
print("Cantidad de pasos: ",cantidadPasos)
set_config(display='diagram')
pipe

Cantidad de pasos:  2


/home/diego/UPS/MachineLearning/Unidad 2/Practica04_RandonForest_SVM/mlvenv/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1624: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('prepcn',
                 ColumnTransformer(n_jobs=2, remainder='passthrough',
                                   transformers=[('catnom',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sex'])])),
                ('prepminmax',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tranminmax',
                                                  Pipeline(steps=[('minmax',
                                                                   MinMaxScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12])]))])

### Preparación modelo SVM optimizado

In [10]:

svm_modelo_optimizado = SVC(kernel='rbf', C=100, gamma=1)
pipe.steps.append(['modelSVMOptimized', svm_modelo_optimizado])
print("Cantidad de pasos: ",len(pipe.steps))
pipe

Cantidad de pasos:  3


Pipeline(steps=[('prepcn',
                 ColumnTransformer(n_jobs=2, remainder='passthrough',
                                   transformers=[('catnom',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sex'])])),
                ('prepminmax',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tranminmax',
                                                  Pipeline(steps=[('minmax',
                                                                   MinMaxScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12])])),
                ['modelSVMOptimized', SVC(C=100, gamma=1)]])

In [11]:
pipe.fit(X_train, y_train)
print("modelo SVM listo")

modelo SVM listo


### Predicción con SVM 

In [12]:
def predecirNuevoPacienteSVM(age='86', anaemia=1, creatinine_phosphokinase=682, diabetes=1,
                            ejection_fraction=45, high_blood_pressure=1, platelets=310000, serum_creatinine=4, 
                            serum_sodium=135, sex='female', smoking=1, time=90):   
     
    cnames=['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex' ,'smoking', 'time']
    Xnew=[age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction, high_blood_pressure, platelets, serum_creatinine, serum_sodium, sex, smoking, time]
    Xnew_Dataframe = pd.DataFrame(data=[Xnew],columns=cnames)
    
    
    pred = (pipe.predict(Xnew_Dataframe) > 0.5).astype("int32")
    pred = pred.flatten()[0]# de 2D a 1D
    return pred

In [13]:
prediccion = predecirNuevoPacienteSVM()
print(f"Prediccion : {prediccion}")
if(prediccion == 0):
    print("El paciente sobrevive")
else:
    print("El paciente fallece")

Prediccion : 1
El paciente fallece


### Explicación de la SVM y las predicciones

La clasificacion o prediccion iniciara con el preprocesamiento de los datos, que incluye la normalización de características como 'age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine' y 'serum_sodium' para asegurar que todas las variables estén en la misma escala. Además, de la codificacion de la variables categórica 'sex', como variables numéricas binaria.



#### Superficies de decisión

Las superficies de decisión son límites en un espacio dimensional que separan diferentes clases o categorías en un problema de clasificación, determinadas por modelos como SVM

